## Peptídeos de Penetração Hematoencefálica

A barreira hematoencefálica (BBB) é uma barreira física altamente complexa que determina quais substâncias podem entrar no cérebro. Neste notebook será apresentado um algoritmo genético para prever as moléculas que penetram a barreira hematoencefálica, o qual é construído utilizando a biblioteca DEAP. As informações sobre estas moléculas estão disponíveis na base de dados <b>descriptors.xlsx</b>, onde cada linha consiste em uma molécula e as colunas correspondem aos descritores destas moléculas.<br>
### Índice
...

### Bibliotecas comumente utilizadas ao longo do código

In [1]:
import joblib as jl #fornece utilitários para salvar e carregar objetos
import numpy as np
import pandas as pd
import pathlib as pl #oferece classes que representam caminhos de sistema
                    #de arquivos com semântica apropriada para diferentes sistemas operacionais

#ignora avisos do processamento
import warnings
warnings.filterwarnings('ignore')


### Funções para carregar a base de dados e retornar os dados em Data Frame

In [2]:
def data_frame_from_excel(*args, **kwargs) -> pd.DataFrame:
    return pd.read_excel(*args, **kwargs)


def data_frame_from_csv(*args, **kwargs) -> pd.DataFrame:
    return pd.read_csv(*args, **kwargs)


### Criação de um mapa de memória, onde os arquivos mapeados na memória são usados para acessar pequenos segmentos de arquivos grandes no disco, sem ler o arquivo inteiro na memória. 

In [3]:
def array_as_memory_map(array: np.ndarray, *, directory: pl.Path) -> np.memmap:
    if not directory.is_dir():
        directory.mkdir()

    memory_map_path = pl.Path(directory) / "memory_map"

    jl.dump(array, memory_map_path)

    return jl.load(memory_map_path, mmap_mode="r")


### Silhouette scores
O valor do coeficiente de silhueta é um valor de -1 a 1, onde um valor alto indica que o objeto está bem combinado com seu próprio cluster e mal combinado com os clusters vizinhos. Se a maioria dos objetos tiver um valor alto, a configuração de cluster é apropriada. Se muitos pontos tiverem um valor baixo ou negativo, a configuração de cluster pode ter muitos ou poucos clusters, ou seja, uma configuração de clusters diferente do apresentado.

In [4]:
#score

from sklearn import cluster, metrics


def get_silhouette_scores(data_frame: pd.DataFrame, *, parallel: jl.Parallel = None) -> pd.DataFrame:
    columns = [data_frame.loc[:, [column]] for column in data_frame.columns]

    if parallel is not None:
        scores = parallel(jl.delayed(__get_silhouette_score__)(column) for column in columns)
    else:
        scores = [__get_silhouette_score__(column) for column in columns]

    index = pd.Index(data_frame.columns.values, name="Feature")
    columns = pd.Index(("Silhouette score",), name="Measure")

    return pd.DataFrame(scores, index=index, columns=columns)


def evaluate_k_means_silhouette(individual: np.ndarray, *, descriptors: np.memmap, final_dim: int) -> tuple[float]:
    if individual.sum() != final_dim:
        return -1.,

    selected_descriptors = descriptors[:, individual]

    return __get_silhouette_score__(selected_descriptors),


# noinspection PyUnresolvedReferences
def __get_silhouette_score__(data_frame: pd.DataFrame) -> float:
    return metrics.silhouette_score(data_frame, cluster.KMeans(n_clusters=2).fit(data_frame).labels_)


### Pré-processamento

A biblioteca <b>Feature-engine</b> possui vários transformadores para projetar e selecionar recursos para usar em modelos de aprendizado de máquina.<br><br>
Este mecanismo de recursos hospeda os seguintes grupos de transformadores:
* Falta de imputação de dados
* Codificação categórica
* Discretização de variável
* Limitação ou remoção de outlier
* Transformação numérica
* Criação de variável
* Seleção de variável
* Recursos de data e hora
* Previsão
* Pré-processando
------------------------------------------------------
Devido ao Python ter que determinar o tipo de objetos durante o tempo de execução, às vezes fica muito difícil para os desenvolvedores descobrirem exatamente o que está acontecendo no código.<br>
O Python tenta mitigar esse problema introduzindo anotação de tipos, com a biblioteca <b>typing</b>, para ajudar os verificadores de tipo externos a identificar quaisquer erros.

In [5]:
import typing as t
from feature_engine import selection

# Apaga atributos iguais
def drop_constant_features(data_frame: pd.DataFrame, *, tolerance: float) -> pd.DataFrame:
    return selection.DropConstantFeatures(tol=tolerance).fit_transform(data_frame)

# Apaga atributos correlacionados
def drop_correlated_features(
        data_frame: pd.DataFrame, sorted_scores: pd.DataFrame, *, tolerance: float) -> pd.DataFrame:
    flagged_feature_names = __get_correlated_feature_names__(data_frame, tolerance=tolerance)

    print(f"Correlated features found: {len(flagged_feature_names)}.")

    while len(flagged_feature_names) > 0:
        print(f"Remaining correlated features: {len(flagged_feature_names)}.", end="\r")

        worst_performing_column_name = sorted_scores.loc[flagged_feature_names, :].index.values[0]
        data_frame = data_frame.drop(worst_performing_column_name, axis=1)
        flagged_feature_names = __get_correlated_feature_names__(data_frame, tolerance=tolerance)

    print("All correlated features removed.")

    return data_frame

# Obtém os nomes dos atributos correlacionados
def __get_correlated_feature_names__(data_frame: pd.DataFrame, *, tolerance: float) -> tuple[t.Any, ...]:
    correlation_matrix = data_frame.corr(method="spearman").abs()
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool_))

    return tuple(column for column in upper_triangle.columns if any(upper_triangle[column] > tolerance))


### Otimização
<b>DEAP</b> (Distributed Evolutionary Algorithms in Python, ou Algoritmos Evolutivos Distribuídos em Python) é uma nova estrutura de computação evolutiva para prototipagem rápida e teste de ideias. Ele procura tornar os algoritmos explícitos e as estruturas de dados transparentes.<br>
Submodulos para a criação do algoritmo genético e suas funções
* <b>creator:</b> criar indivíduos e a função de aptidão (fitness)
* <b>base:</b> registrar os elementos dos algoritmos genéticos, vincula os elementos às bases de dados
* <b>tools:</b> permite utilizar a função dos operadores (mutação, cruzamento)
* <b>algorithms:</b> executa o algoritmo genético

In [6]:
from deap import algorithms, base, creator, tools

# função que registra os abjetivos e elementos do algoritmo genético
def get_toolbox(*, descriptors: np.memmap, final_dim: int, 
                ind_size: int, pop_size: int, parallel: jl.Parallel = None) -> base.Toolbox:
    
    # weights = 1.0, pois é um problema de maximização
    creator.create("Score", base.Fitness, weights=(1.0,))
    
    #o individuo é criado no formato ndarray
    creator.create("Individual", np.ndarray, dtype="bool", fitness=creator.Score)
    
    #objeto onde serão salvos os elementos do AG
    toolbox = base.Toolbox()

    #registrando elementos do AG
    toolbox.register("individual", create_individual, size=ind_size, final_dim=final_dim)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=pop_size)
    toolbox.register("evaluate", evaluate_k_means_silhouette, descriptors=descriptors, final_dim=final_dim)
    toolbox.register("mate", tools.cxPartialyMatched)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    return toolbox


# criar individuos
def create_individual(size: int, *, final_dim: int):
    array = np.zeros(size, dtype=np.bool_)
    random_indices = np.random.randint(0, size, size=final_dim)
    array[random_indices] = np.True_
    
    return creator.Individual(array)


def joblib_map(function: t.Callable, *arg_lists, parallel: jl.Parallel):
    return parallel(jl.delayed(function)(*args) for args in zip(*arg_lists))

# o algoritmo genético será criado conforme os elementos definidos nos registros
def genetic_algorithm(toolbox: base.Toolbox) -> list[np.ndarray]:
    population = toolbox.population()
    
    #melhor dimensao de dados e melhor fitness são inicializados
    best_dim, best_fit = 0, 0

    #geracao e geracao sem progresso são inicializados
    gen, gens_without_progress = 0, 0
    
    while gens_without_progress < 100:  # enquanto genes sem progresso for menor que 100
        gen += 1
        print_prefix = f"[{gen} ({gens_without_progress} without progress) | " \ #mostra a quantidade de genes sem progresso
                       f"Best individual: (D = {best_dim}, F = {best_fit})]"     #e o melhor fitness encontrado

        print(f"{print_prefix} Selecting parents...", end="\r")
        
        # seleciona os pais, faz cruzamento e mutação destes
        offspring = toolbox.select(population, k=len(population) // 2) + toolbox.population(n=len(population) // 2)
        print(f"{print_prefix} Mating and mutating selected parents...", end="\r")
        
        #gera pontuação da aptidão dos filhos
        offspring = algorithms.varAnd(offspring, toolbox, cxpb=0.5, mutpb=0.2) # cxpb = prob. de cruzamento; mutpb = prob. de mutação
        print(f"{print_prefix} Scoring offspring fitness...", end="\r")
        
        # salva fitness dos filhos
        values = toolbox.map(toolbox.evaluate, offspring)

        # adiciona os fitness aos respectivos filhos
        for value, individual in zip(values, offspring):
            individual.fitness.values = value

        print(f"{print_prefix} Selecting best from parents and children...", end="\r")
        population = toolbox.select(population + offspring, k=len(population))

        #melhor individuo
        best_ind = tools.selBest(population, k=1)
        dim, fit = np.where(best_ind[0] == 1)[0].size, best_ind[0].fitness.values[0]

        #se a dimensão e o melhor indivíduo nao mudarem, a geração é dada como sem progresso
        if dim == best_dim and fit == best_fit:
            gens_without_progress += 1
        else:
        #se a dimensão e o melhor indivíduo melhorar, tem progresso e o num de gerações sem progresso zera
            best_dim, best_fit = dim, fit
            gens_without_progress = 0

    print(f"{print_prefix} Execution finished.")

    #retorna a melhor população
    return tools.selBest(population, k=1)[0]


### Execução



In [7]:
#playground
import pathlib as pl

import joblib as jl

#for i in range(50):
descriptors_file = pl.Path("data/descriptors.xlsx")
output_directory = pl.Path("results")#+str(i))

if not output_directory.is_dir():
    output_directory.mkdir()

variance_threshold, correlation_threshold = .95, .7
n_jobs = -1
final_dimensionality = 5
population_size = 100


with jl.Parallel(n_jobs=n_jobs) as parallel:
    if not (output_directory / "final_descriptors.csv").is_file():
        descriptors = data_frame_from_excel(
            descriptors_file, sheet_name="Blad1", header=2, index_col=0
        ).reset_index(drop=True)
        filtered_descriptors = drop_constant_features(descriptors, tolerance=variance_threshold)
        scores = get_silhouette_scores(filtered_descriptors, parallel=parallel)
        final_descriptors = drop_correlated_features(
            filtered_descriptors, scores.sort_values(by="Silhouette score"), tolerance=correlation_threshold)
        final_descriptors.to_csv(output_directory / "final_descriptors.csv", sep=";", index=False)
    else:
        final_descriptors = data_frame_from_csv(output_directory / "final_descriptors.csv", sep=";")

    data_memory_map = array_as_memory_map(final_descriptors.to_numpy(), directory=output_directory)

    toolbox = get_toolbox(
        descriptors=data_memory_map,
        final_dim=final_dimensionality,
        ind_size=final_descriptors.shape[1],
        pop_size=population_size,
        parallel=parallel
    )

    best_solution = genetic_algorithm(toolbox)
    final_descriptors.iloc[:, best_solution].to_csv(output_directory / "selected_descriptors.csv", sep=";", index=False)

    with open(output_directory / "selected_descriptors_fitness.txt", "w", encoding="utf-8") as file:
        # noinspection PyUnresolvedReferences
        file.write(str(best_solution.fitness.values[0]))


[132 (99 without progress) | Best individual: (D = 5, F = 0.9877497906071292)] Execution finished. parents and children...
